In [1]:
from bs4 import BeautifulSoup
import urllib

rawDataFile = open('tarrant_county_raw.csv', 'r')

start = True
rawDataLines = []
for line in rawDataFile.readlines():
    if (not start):
        line = line.replace('\n', '')
        line = line.replace('\xff', '')
        line = line.strip()
        variables = line.split(',')
        if (len(variables) == 7):
            variables[5] = variables[5] + variables[6]
            variables[5] = variables[5].replace('$', '')
        variables.pop()
        rawDataLines.append(variables)
    start = False
    
rawDataFile.close()

In [2]:
import time
from datetime import datetime

fileName = "tarrant_county_" + datetime.now().strftime("%m_%d_%Y_%M_%S") + ".txt"

parsedDataFile = open(fileName, 'w')

parsedDataFile.write('id;address;land_market_share;impr_market_share;total_market_share;land_appraisal_value;impr_appraisal_value;total_appraisal_value;land_approximate_size;impr_approximate_size;total_approximate_size;land_land_acres;impr_land_acres;total_land_acres;land_land_sqft;impr_land_sqft;total_land_sqft' + '\n')

for rawDataLine in rawDataLines:
    try:
        # Get the html and parse it using beautiful soup
        r = urllib.urlopen('http://www.tad.org/property-data-sheet-residential?keyword=' + rawDataLine[0]).read()
        soup = BeautifulSoup(r, 'html.parser')

        line = ''
        
        line = rawDataLine[0] + ";"
        
        # Isolate the home's address
        tables = soup.findAll("table")
        tableRows = tables[0].findAll("tr")
        tableCells = tableRows[2].findAll("td")
        line = line + tableCells[1].text.strip() + ';'

        dataRows = tables[1].findAll('tr')
        cells = []
        for row in dataRows:
            dataCells = row.findAll('td')
            for dataCell in dataCells:
                cells.append(dataCell.text.strip())

        indexes = [5,6,7,9,10,11,13,14,15,17,18,19,21,22,23]

        for index in indexes:
            if (cells[index] != ''):
                line = line + cells[index].replace('$', '').replace(',', '').strip() + ';'
            else:
                line = line + 'NULL' + ';'
        line = line[:-1]
        parsedDataFile.write(line + '\n')
    except:
        print("Error with record: " + rawDataLine[0])
        continue
    time.sleep(2)
    
parsedDataFile.close()

In [3]:
headers = open("tarrant_county_04_27_2017_04_58.txt", 'r').readlines()[0]
index = 1
while (index < 5):
    with open("tarrant_county_04_27_2017_04_58_" + str(index) + ".txt", 'a') as outfile:
        outfile.write(headers)
    index = index + 1
index = 1
for row in open("tarrant_county_04_27_2017_04_58.txt", 'r').readlines():
    with open("tarrant_county_04_27_2017_04_58_" + str(index) + ".txt", 'a') as outfile:
        outfile.write(row)
    if (index == 4):
        index = 0
    index = index + 1

In [4]:
import time
import zillow
import pprint
import json
import os.path
from datetime import datetime

pp = pprint.PrettyPrinter(indent=4)
firstLoop = True
headers = []

if not os.path.exists('zillow_address_found.txt'):
    open('zillow_address_found.txt', 'w').close()

def convertRowToDict(headers, dataParts):
    index = 0
    dataDict = {}
    for header in headers:
        header = header.replace('\n', '').strip()
        dataDict[header] = dataParts[index].replace('\n', '').strip()
        index = index + 1
    return dataDict

fileName = "full_data_set_" + datetime.now().strftime("%m_%d_%Y_%M_%S") + ".txt"

with open(fileName, 'a') as outfile:
    for row in open("tarrant_county_04_27_2017_04_58_4.txt", 'r').readlines():
        detailDataDict = {}
        dataParts = row.split(';')
        if firstLoop:
            headers = dataParts
            firstLoop = False
        else:
            try:
                found = False
                with open('zillow_address_found.txt', 'r') as zillowDataFound:
                    for line in zillowDataFound.readlines():
                        if(dataParts[1].upper() in line.upper()):
                            found = True
                            break
                if not found:
                    addressParts = dataParts[1].split(',')
                    address = addressParts[0].strip() + ', ' + addressParts[1].strip()
                    postalCode = addressParts[2].strip()
                    #key = "X1-ZWz198ccdzns3v_6frtk" # old
                    #key = "X1-ZWz1fsdkwnam17_1pt0f" # new
                    key = "X1-ZWz1fse0ow71u3_211k7" # new new
                    api = zillow.ValuationApi()
                    detailDataDict["zillow"] = api.GetDeepSearchResults(key, address, postalCode).get_dict()
                    detailDataDict["success"] = True
                    with open('zillow_address_found.txt', 'a') as zillowDataFound:
                        zillowDataFound.write(dataParts[1] + '\n')
                else:
                    detailDataDict["success"] = False
            except:
                detailDataDict["success"] = False
            if detailDataDict["success"]:
                detailDataDict["tarrant_county_data"] = convertRowToDict(headers, dataParts)
                json.dump(detailDataDict, outfile)
                outfile.write('\n')
                print(str(detailDataDict["tarrant_county_data"]["address"]) + ", " + str(detailDataDict["success"]))

In [5]:
from os import listdir

with open("final_data_set.txt", 'w') as outfile:
    for directory in listdir("C:/Users/asavv/Google Drive/Andreas Documents/Education/Course Work/INSY 5376 - Big Data Analytics\Project"):
        if "full_data_set_" in directory:
            print(directory)
            for line in open(directory, 'r').readlines():
                outfile.write(line)

In [6]:
import pprint
import json
    
def variableCheck(jsonRecord):
    if jsonRecord is not None:
        return str(jsonRecord).strip()
    else:
        return "NULL"
    
idAdded = []

with open('final_data_set.csv', 'w') as csvfile:
    csvfile.write('id,zpid,address,city,zipcode,total_approximate_size,total_appraisal_value,' +
                  'total_market_share,zestimate_amount,valuation_range_high,valuation_range_low,' + 
                  'bedrooms,bathrooms,usecode,last_sold_date,year_built,complete,finished_sqft\n')
    with open('final_data_set.txt', 'r') as outfile:
        jsonDocs = outfile.readlines()
        for jsonDoc in jsonDocs:
            jsonRecord = json.loads(jsonDoc)
            if jsonRecord["tarrant_county_data"]["id"] not in idAdded:
                variables = []
                variables.append(variableCheck(jsonRecord["tarrant_county_data"]["id"]))
                variables.append(variableCheck(jsonRecord["zillow"]["zpid"]))
                
                for address_line in jsonRecord["tarrant_county_data"]["address"].split(','):
                    variables.append(variableCheck(address_line))
                
                variables.append(variableCheck(jsonRecord["tarrant_county_data"]["total_approximate_size"]))
                variables.append(variableCheck(jsonRecord["tarrant_county_data"]["total_appraisal_value"]))
                variables.append(variableCheck(jsonRecord["tarrant_county_data"]["total_market_share"]))
                
                variables.append(variableCheck(jsonRecord["zillow"]["zestimate"]["amount"]))
                variables.append(variableCheck(jsonRecord["zillow"]["zestimate"]["valuation_range_high"]))
                variables.append(variableCheck(jsonRecord["zillow"]["zestimate"]["valuation_range_low"]))
                
                variables.append(variableCheck(jsonRecord["zillow"]["extended_data"]["bedrooms"]))
                variables.append(variableCheck(jsonRecord["zillow"]["extended_data"]["bathrooms"]))
                
                variables.append(variableCheck(jsonRecord["zillow"]["extended_data"]["usecode"]))
                variables.append(variableCheck(jsonRecord["zillow"]["extended_data"]["last_sold_date"]))
                variables.append(variableCheck(jsonRecord["zillow"]["extended_data"]["year_built"]))
                variables.append(variableCheck(jsonRecord["zillow"]["extended_data"]["complete"]))
                variables.append(variableCheck(jsonRecord["zillow"]["extended_data"]["finished_sqft"]))
                
                line = ''
                for variable in variables:
                    line = line + variable + ','
                line = line.strip(',')
                csvfile.write(line + '\n')
                idAdded.append(jsonRecord["tarrant_county_data"]["id"])
print("Done")